In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from flask import Flask, request, jsonify
import joblib


In [ ]:

# Step 1: Data Preprocessing
# Load the dataset
data = pd.read_csv('customer_churn_large_dataset.csv')

# Handle missing data 
data.fillna(data.mean(), inplace=True)



# Encode categorical variables (if 'Gender' and 'Location' are categorical)
data = pd.get_dummies(data, columns=['Gender', 'Location'])

# Split data into features and target variable
X = data.drop('Churn', axis=1)  
y = data['Churn']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Feature Engineering (if necessary)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 3: Model Building

model = RandomForestClassifier(n_estimators=100, random_state=42)


model.fit(X_train, y_train)

# Step 4: Model Evaluation

y_pred = model.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print performance metrics
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')



In [ ]:
# Step 5: Model Optimization


param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}


grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_params = grid_search.best_params_


optimized_model = RandomForestClassifier(**best_params, random_state=42)
optimized_model.fit(X_train, y_train)


y_pred_optimized = optimized_model.predict(X_test)


accuracy_optimized = accuracy_score(y_test, y_pred_optimized)
precision_optimized = precision_score(y_test, y_pred_optimized)
recall_optimized = recall_score(y_test, y_pred_optimized)
f1_optimized = f1_score(y_test, y_pred_optimized)


print("Optimized Model Performance:")
print(f'Accuracy: {accuracy_optimized}')
print(f'Precision: {precision_optimized}')
print(f'Recall: {recall_optimized}')
print(f'F1-Score: {f1_optimized}')


In [ ]:
# Step 6: Model Deployment


app = Flask(__name__)


loaded_model = joblib.load('model.pkl')


@app.route('/predict', methods=['POST'])
def predict():
    try:
        
        input_data = request.get_json()
        
        
        predictions = loaded_model.predict(input_data)
        
        
        return jsonify({'predictions': predictions.tolist()})
    except Exception as e:
        return jsonify({'error': str(e)})

# Run the Flask application
if __name__ == '__main__':
    app.run(debug=True)
